## Import 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install graphviz;
#!pip uninstall dtreeviz

In [ ]:
## look into the dater
#from fastbook import *
#from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
#!pip install dtreeviz
#from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split

In [ ]:
## pandas
pd.options.display.max_rows = 20
pd.options.display.max_columns = 12

In [ ]:
main_path = "../input/ieee-fraud-detection-joined-tables/"

In [ ]:
df = pd.read_csv(main_path+"tr_sample-100k.csv")
df_te= pd.read_csv(main_path + "te_tran_iden.csv")

In [ ]:
## ratio of true and false
df.isFraud.value_counts()[1]/df.isFraud.value_counts()[0] *100

In [ ]:
df = df.sample(50000, random_state=42)
#df_te = df_te.sample(50000, random_state=42)
df.isFraud.value_counts()[1]/df.isFraud.value_counts()[0] *100

## Check Dater 

In [ ]:
## check tables for missing values in join column
col = "shape", "columns with nans ", "nans in TransactioniD"
ind = "train","test"
lst1 = [df.shape, sum(df.isna().sum()>0), df.TransactionID.isna().sum()]
lst2 = [df_te.shape, sum(df_te.isna().sum()>0), df_te.TransactionID.isna().sum()]
pd.DataFrame([lst1,lst2], columns=col,index=ind)

In [ ]:
## All colaths from test in train
set(df_te.columns).issubset(set(df.columns))

In [ ]:
## Class imbalance? Might need to upsample or downsample???? Isn't usage of false positves and true positives overcoming imbalance?
sum(df["isFraud"]==True)/len(df)

## Trimming col from the dater

In [ ]:
## remove columns with nans>300k
##[((df.isna().sum()>n/2*1e5).sum(),n/2) for n in range(1,13)]
#to_drop = df.columns[df.isna().sum()>300000].to_list()
to_drop=[] # forcing to use all variables
df.drop(to_drop,1, inplace=True)
df.shape

## Tabular Pandas

In [ ]:
## sort by time
df.sort_values("TransactionDT", inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
## split to train and valid ind linearly
msk = np.arange(len(df))<0.8*len(df)
splits = (list(np.where(msk)[0]),list(np.where(~msk)[0]))
len(splits[0])+len(splits[1]), len(df), len(splits[0])

In [ ]:
## variables for TP
dep_var = "isFraud"
procs = [Categorify,FillMissing]
## Cont and Cat variables
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)

In [ ]:
## check if cont variables in test have nans where train doesn't
#def na_check(col): return df.loc[:,col].isna().sum()>0,df_te.loc[:,col].isna().sum()>0
#na_col = {col: na_check(col) for col in cont}
na_col_te = [col for col in cont if df.loc[:,col].isna().sum()==0 and df_te.loc[:,col].isna().sum()>0]
na_col_te


In [ ]:
## Add a last row with nans for col:na_col_te
#df.insert(0,df.columns,df.median())

df = pd.concat([df.median().to_frame().T, df], ignore_index=True)

df.loc[df.index[0],na_col_te] = np.nan
df.loc[df.index[0],na_col_te]
#df.median()
#df

In [ ]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)
len(to.train)+len(to.valid)

In [ ]:
to.items.columns

In [ ]:
to.valid.show(3)

In [ ]:
to.items.head()

In [ ]:
##vocab
to.classes["ProductCD"]

In [ ]:
save_pickle("to.pkl",to)

In [ ]:
load_pickle("to.pkl")

## Model

In [ ]:
def rf(xs, y, n_estimators=40, max_samples=20_000,#200_000,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestClassifier(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y
xs.shape

In [ ]:
#m = DecisionTreeRegressor(max_leaf_nodes=4)
#m.fit(xs, y);
m = rf(xs,y)


## Output of Test dater

Outputs that are interesting are model accuracy, maybe f1score?, model ROC,AOC?-

In [ ]:
## Metrics functions
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)
def m_rmse_p(m, xs, y): return r_mse(m.predict_proba(xs), y)
#def m_accuracy(m,xs,y): return sum(m.predict(xs)==y)/len(xs)
def m_accuracy2(m,xs,y): return m.score(xs,y)
def m_conf(m,xs,y): return confusion_matrix(y, m.predict(xs))

In [ ]:
## See predicted column. Does it match expectations
pd.DataFrame(m.predict(xs)).describe()

In [ ]:
## Accuracy
m_accuracy2(m, xs, y), m_accuracy2(m, valid_xs, valid_y)

In [ ]:
## Rmse (probably not useful)
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

In [ ]:
## Confusion Matrix
m_conf(m,xs,y), m_conf(m,valid_xs,valid_y)

In [ ]:
m.predict_proba(valid_xs)

In [ ]:
from sklearn.metrics import plot_roc_curve
plot_roc_curve(m,valid_xs,valid_y)

In [ ]:
plot_roc_curve(m,xs,y)

# ## predict on test dater

In [ ]:
## check rows with nan values and compare with train dset

In [ ]:
## drop from test dater
df_te.drop(to_drop,1, inplace=True)
df_te.shape

In [ ]:
## make new Tabular Pandas for test dataset
to_tst = to.new(df_te)
to_tst.process()

In [ ]:
## "isFraud" is a cont variable?
# to_classes["isFraud"]

In [ ]:
to.classes["ProductCD"], to_tst.classes["ProductCD"]

In [ ]:
to.items.isFraud.astype("category").describe()

In [ ]:
## make predictions 
test_xs = to_tst.xs
pred = m.predict_proba(test_xs)
pred[:,1]#1 is the class we are interested in, i.e, 0,1...

In [ ]:
pd.DataFrame(pred).describe()

In [ ]:
m.classes_[1]

In [ ]:
## Make into submission dataframe
lst_pred = [df_te["TransactionID"].to_list(),pred[:,1]]
len(lst_pred)
df_pred = pd.DataFrame({"TransactionID":lst_pred[0],"isFraud":lst_pred[1]})
df_pred

In [ ]:
df_pred.to_csv("my_subs.csv",index=False)

## Resources
More info on the dater
1. https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203 
1. [Notebook on RF](https://www.kaggle.com/raviolli77/random-forest-in-python#Training-Algorithm)
2. [searching hyperparameters](https://www.kaggle.com/raviolli77/random-forest-in-python#Creating-Training-and-Test-Sets)